In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set styling
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Data paths
DATA_DIR = Path("/Users/dinokage/dev/fazri-analyzer/backend/augmented")

print("📊 Campus Entity Resolution - Dataset Analysis")
print("="*60)

📊 Campus Entity Resolution - Dataset Analysis


In [28]:
def load_datasets():
    """Load all CSV datasets and return dict"""
    datasets = {}
    
    csv_files = {
        'profiles': 'student_staff_profiles.csv',
        'cctv': 'cctv_frames_augmented.csv',
        'swipes': 'campus_card_swipes_augmented.csv', 
        'wifi': 'wifi_associations_logs_augmented.csv',
        'library': 'library_checkouts_augmented.csv',
        'bookings': 'lab_bookings_augmented.csv',
        'helpdesk': 'helpdesk_augmented.csv',
        'face_embeddings': 'face_embeddings.csv'
    }
    
    for name, filename in csv_files.items():
        filepath = DATA_DIR / filename
        if filepath.exists():
            datasets[name] = pd.read_csv(filepath)
            print(f"✅ Loaded {name}: {len(datasets[name])} rows")
        else:
            print(f"❌ Missing {name}: {filename}")
    
    return datasets

# Load all data
data = load_datasets()

✅ Loaded profiles: 7000 rows
✅ Loaded cctv: 28000 rows
✅ Loaded swipes: 32000 rows
✅ Loaded wifi: 32000 rows
✅ Loaded library: 28000 rows
✅ Loaded bookings: 28000 rows
✅ Loaded helpdesk: 28000 rows
✅ Loaded face_embeddings: 6973 rows


In [29]:
# Cell 3: Profile Data Analysis
print("\n🔍 STUDENT/STAFF PROFILES ANALYSIS")
print("="*60)

if 'profiles' in data:
    df = data['profiles']
    
    print(f"\nTotal Records: {len(df)}")
    print(f"\nColumns: {df.columns.tolist()}")
    print(f"\nData Types:\n{df.dtypes}")
    print(f"\nFirst 5 Records:\n{df.head(200)}")
    
    # Identify all identifier columns
    id_columns = [col for col in df.columns if 'id' in col.lower() 
                  or col.lower() in ['email', 'card_id', 'device_hash', 'face_id', 'entity_id']]
    print(f"\n📋 Identifier Columns: {id_columns}")
    
    # Check for missing values in identifiers
    print(f"\n⚠️ Missing Values in Identifiers:")
    for col in id_columns:
        if col in df.columns:
            missing = df[col].isna().sum()
            missing_pct = (missing / len(df)) * 100
            print(f"  - {col}: {missing} ({missing_pct:.1f}%)")
    
    # Check for duplicates
    print(f"\n🔄 Duplicate Analysis:")
    for col in id_columns:
        if col in df.columns:
            duplicates = df[col].duplicated().sum()
            print(f"  - {col}: {duplicates} duplicates")
    
    # Name variations analysis
    if 'name' in df.columns:
        print(f"\n📝 Name Variations:")
        name_counts = df['name'].value_counts()
        print(f"  - Unique names: {len(name_counts)}")
        print(f"  - Names appearing >1 time: {(name_counts > 1).sum()}")


🔍 STUDENT/STAFF PROFILES ANALYSIS

Total Records: 7000

Columns: ['entity_id', 'name', 'role', 'email', 'department', 'student_id', 'staff_id', 'card_id', 'device_hash', 'face_id']

Data Types:
entity_id      object
name           object
role           object
email          object
department     object
student_id     object
staff_id       object
card_id        object
device_hash    object
face_id        object
dtype: object

First 5 Records:
    entity_id          name     role               email department  \
0     E100000    Neha Mehta  student    user0@campus.edu      CIVIL   
1     E100001    Neha Kumar  student    user1@campus.edu    Physics   
2     E100002    Neha Singh  student    user2@campus.edu      Admin   
3     E100003  Ishaan Desai  student    user3@campus.edu      Admin   
4     E100004   Rohan Desai  student    user4@campus.edu       MECH   
..        ...           ...      ...                 ...        ...   
195   E100195  Rohan Sharma  student  user195@campus.edu

In [ ]:
# Cell 4: Card Swipe Analysis
print("\n🔍 CARD SWIPE LOGS ANALYSIS")
print("="*60)

if 'swipes' in data:
    df = data['swipes']
    
    print(f"\nTotal Swipes: {len(df)}")
    print(f"\nColumns: {df.columns.tolist()}")
    print(f"\nSample Data:\n{df.head(20)}")
    
    # Convert timestamp
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Temporal analysis
    print(f"\n📅 Temporal Coverage:")
    print(f"  - Start: {df['timestamp'].min()}")
    print(f"  - End: {df['timestamp'].max()}")
    print(f"  - Duration: {(df['timestamp'].max() - df['timestamp'].min()).days} days")
    
    # Location analysis
    if 'location_id' in df.columns:
        print(f"\n📍 Location Distribution:")
        print(df['location_id'].value_counts().head(10))
    
    # Unique entities
    if 'card_id' in df.columns:
        print(f"\n👥 Unique Cards: {df['card_id'].nunique()}")
        
    # Activity patterns
    df['hour'] = df['timestamp'].dt.hour
    print(f"\n⏰ Peak Activity Hours:")
    print(df['hour'].value_counts().sort_index().head(10))

In [ ]:
# Cell 5: Wi-Fi Logs Analysis
print("\n🔍 WI-FI LOGS ANALYSIS")
print("="*60)

if 'wifi' in data:
    df = data['wifi']
    
    print(f"\nTotal Wi-Fi Connections: {len(df)}")
    print(f"\nColumns: {df.columns.tolist()}")
    print(f"\nSample Data:\n{df.head(20)}")
    
    # Convert timestamp
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Device analysis
    if 'device_hash' in df.columns:
        print(f"\n📱 Unique Devices: {df['device_hash'].nunique()}")
    
    # Access point analysis
    if 'ap_id' in df.columns:
        print(f"\n📡 Access Points:")
        print(df['ap_id'].value_counts().head(10))
    
    # Temporal patterns
    df['hour'] = df['timestamp'].dt.hour
    print(f"\n⏰ Connection Patterns by Hour:")
    hourly = df.groupby('hour').size()
    print(hourly.head(10))

In [ ]:
# Cell 6: Library Checkouts Analysis
print("\n🔍 LIBRARY CHECKOUTS ANALYSIS")
print("="*60)

if 'library' in data:
    df = data['library']
    
    print(f"\nTotal Checkouts: {len(df)}")
    print(f"\nColumns: {df.columns.tolist()}")
    print(f"\nSample Data:\n{df.head(20)}")
    
    # Identifier analysis
    id_cols = [col for col in df.columns if 'id' in col.lower() or col.lower() in ['timestamp']]
    print(f"\n🆔 Identifiers: {id_cols}")
    
    # Most borrowed items
    if 'item_id' in df.columns or 'book_id' in df.columns:
        item_col = 'item_id' if 'item_id' in df.columns else 'book_id'
        print(f"\n📚 Most Borrowed Items:")
        print(df[item_col].value_counts().head(10))

In [ ]:
# Cell 7: Helpdesk Notes Analysis
print("\n🔍 HELPDESK NOTES ANALYSIS")
print("="*60)

if 'helpdesk' in data:
    df = data['helpdesk']
    
    print(f"\nTotal Tickets: {len(df)}")
    print(f"\nColumns: {df.columns.tolist()}")
    print(f"\nSample Data:\n{df.head(2000)}")
    
    # Text analysis
    if 'notes' in df.columns or 'description' in df.columns:
        text_col = 'notes' if 'notes' in df.columns else 'description'
        print(f"\n📝 Text Statistics:")
        print(f"  - Avg length: {df[text_col].str.len().mean():.0f} chars")
        print(f"  - Max length: {df[text_col].str.len().max()} chars")
        
        # Sample text
        print(f"\n📄 Sample Text:")
        print(df[text_col].iloc[0][:200] + "...")
    
    # Check for embedded identifiers in text
    print(f"\n🔍 Checking for identifiers in text...")
    if 'notes' in df.columns or 'description' in df.columns:
        text_col = 'notes' if 'notes' in df.columns else 'description'
        
        # Look for patterns
        import re
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        id_pattern = r'\b[A-Z]{2,}\d{4,}\b'
        
        emails = df[text_col].str.extract(f'({email_pattern})', expand=False).dropna()
        ids = df[text_col].str.extract(f'({id_pattern})', expand=False).dropna()
        
        print(f"  - Found {len(emails)} potential emails")
        print(f"  - Found {len(ids)} potential IDs")

In [ ]:
# Cell 8: Face Embeddings Analysis
print("\n🔍 FACE EMBEDDINGS ANALYSIS")
print("="*60)

if 'face_embeddings' in data:
    df = data['face_embeddings']
    
    print(f"\nTotal Embeddings: {len(df)}")
    print(f"\nColumns: {df.columns.tolist()}")
    print(f"\nSample Data:\n{df.head(20)}")
    
    # Check embedding dimensions
    embedding_cols = [col for col in df.columns if 'embed' in col.lower() or col.startswith('dim_')]
    print(f"\n🎯 Embedding Dimensions: {len(embedding_cols)}")
    
    # Unique faces
    if 'face_id' in df.columns or 'person_id' in df.columns:
        id_col = 'face_id' if 'face_id' in df.columns else 'person_id'
        print(f"\n👤 Unique Faces: {df[id_col].nunique()}")

In [ ]:
# Cell 9: Cross-Dataset Identifier Mapping
print("\n🔍 CROSS-DATASET IDENTIFIER MAPPING")
print("="*60)

# Build identifier co-occurrence matrix
identifier_types = {
    'student_id': [],
    'entity_id': [],
    'staff_id': [],
    'email': [],
    'card_id': [],
    'device_hash': [],
    'face_id': []
}

# Check which datasets have which identifiers
for dataset_name, df in data.items():
    print(f"\n{dataset_name.upper()}:")
    for id_type in identifier_types.keys():
        if id_type in df.columns:
            count = df[id_type].notna().sum()
            unique = df[id_type].nunique()
            print(f"  ✅ {id_type}: {count} records, {unique} unique")
            identifier_types[id_type].append(dataset_name)
        else:
            print(f"  ❌ {id_type}: not present")

# Summary
print(f"\n📊 IDENTIFIER AVAILABILITY SUMMARY:")
print("="*60)
for id_type, datasets in identifier_types.items():
    if datasets:
        print(f"{id_type}: Found in {len(datasets)} datasets - {', '.join(datasets)}")
    else:
        print(f"{id_type}: Not found in any dataset")

In [ ]:
# Cell 10: Data Quality Assessment
print("\n🔍 DATA QUALITY ASSESSMENT")
print("="*60)

quality_report = {}

for name, df in data.items():
    print(f"\n{name.upper()}:")
    
    # Missing values
    missing = df.isna().sum()
    missing_pct = (missing / len(df)) * 100
    
    if missing.sum() > 0:
        print(f"  ⚠️ Columns with missing data:")
        for col, pct in missing_pct[missing_pct > 0].items():
            print(f"    - {col}: {pct:.1f}%")
    else:
        print(f"  ✅ No missing data")
    
    # Duplicates
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        print(f"  ⚠️ Duplicate rows: {duplicates}")
    else:
        print(f"  ✅ No duplicate rows")
    
    quality_report[name] = {
        'total_rows': len(df),
        'missing_values': missing.sum(),
        'duplicates': duplicates
    }

# Overall quality score
print(f"\n📊 OVERALL DATA QUALITY:")
total_rows = sum(r['total_rows'] for r in quality_report.values())
total_missing = sum(r['missing_values'] for r in quality_report.values())
total_duplicates = sum(r['duplicates'] for r in quality_report.values())

completeness = (1 - total_missing / (total_rows * len(data))) * 100
print(f"  - Completeness: {completeness:.1f}%")
print(f"  - Total duplicates: {total_duplicates}")

In [ ]:
# Cell 11: Generate Analysis Summary Report
print("\n📝 GENERATING ANALYSIS SUMMARY REPORT")
print("="*60)

# Helper function to convert numpy/pandas types to native Python types
def convert_to_json_serializable(obj):
    """Convert numpy/pandas types to JSON-serializable Python types"""
    if isinstance(obj, dict):
        return {k: convert_to_json_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_json_serializable(item) for item in obj]
    elif isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    else:
        return obj

report = {
    'dataset_overview': {
        'total_datasets': len(data),
        'total_records': sum(len(df) for df in data.values()),
        'datasets': {name: len(df) for name, df in data.items()}
    },
    'identifiers': {},
    'temporal_coverage': {},
    'data_quality': {}
}

# Convert quality report to JSON-serializable format
for name, stats in quality_report.items():
    report['data_quality'][name] = {
        'total_rows': int(stats['total_rows']),
        'missing_values': int(stats['missing_values']),
        'duplicates': int(stats['duplicates'])
    }

# Identifier summary
for id_type, datasets in identifier_types.items():
    if datasets:
        report['identifiers'][id_type] = {
            'available_in': datasets,
            'count': len(datasets)
        }

# Add temporal coverage if available
if 'swipes' in data:
    df = data['swipes'].copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    report['temporal_coverage']['swipes'] = {
        'start_date': df['timestamp'].min().isoformat(),
        'end_date': df['timestamp'].max().isoformat(),
        'duration_days': int((df['timestamp'].max() - df['timestamp'].min()).days)
    }

if 'wifi' in data:
    df = data['wifi'].copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    report['temporal_coverage']['wifi'] = {
        'start_date': df['timestamp'].min().isoformat(),
        'end_date': df['timestamp'].max().isoformat(),
        'duration_days': int((df['timestamp'].max() - df['timestamp'].min()).days)
    }

# Convert entire report to ensure all values are JSON serializable
report = convert_to_json_serializable(report)

# Save report
import json
report_path = Path("../../data/analysis/dataset_analysis_report.json")
report_path.parent.mkdir(parents=True, exist_ok=True)

with open(report_path, 'w') as f:
    json.dump(report, f, indent=2, default=str)  # Added default=str as fallback

print(f"✅ Report saved to: {report_path}")
print(f"\n📊 Analysis Summary:")
print(json.dumps(report, indent=2))

# Also save a human-readable markdown report
md_report_path = Path("../../data/analysis/dataset_analysis_report.md")

with open(md_report_path, 'w') as f:
    f.write("# Dataset Analysis Report\n\n")
    f.write(f"**Generated:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    f.write("## Dataset Overview\n\n")
    f.write(f"- **Total Datasets:** {report['dataset_overview']['total_datasets']}\n")
    f.write(f"- **Total Records:** {report['dataset_overview']['total_records']:,}\n\n")
    
    f.write("### Records per Dataset\n\n")
    f.write("| Dataset | Records |\n")
    f.write("|---------|--------:|\n")
    for name, count in report['dataset_overview']['datasets'].items():
        f.write(f"| {name} | {count:,} |\n")
    
    f.write("\n## Identifier Mapping\n\n")
    f.write("| Identifier | Available In | Dataset Count |\n")
    f.write("|------------|--------------|---------------:|\n")
    for id_type, info in report['identifiers'].items():
        datasets_str = ', '.join(info['available_in'])
        f.write(f"| {id_type} | {datasets_str} | {info['count']} |\n")
    
    f.write("\n## Data Quality\n\n")
    f.write("| Dataset | Total Rows | Missing Values | Duplicates |\n")
    f.write("|---------|------------|----------------|------------|\n")
    for name, stats in report['data_quality'].items():
        f.write(f"| {name} | {stats['total_rows']:,} | {stats['missing_values']:,} | {stats['duplicates']:,} |\n")
    
    if report['temporal_coverage']:
        f.write("\n## Temporal Coverage\n\n")
        for dataset, coverage in report['temporal_coverage'].items():
            f.write(f"### {dataset.title()}\n")
            f.write(f"- Start: {coverage['start_date']}\n")
            f.write(f"- End: {coverage['end_date']}\n")
            f.write(f"- Duration: {coverage['duration_days']} days\n\n")

print(f"✅ Markdown report saved to: {md_report_path}")

In [ ]:
# Cell 12: Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Campus Data Overview', fontsize=16, fontweight='bold')

# 1. Dataset sizes
if data:
    dataset_sizes = {name: len(df) for name, df in data.items()}
    axes[0, 0].bar(dataset_sizes.keys(), dataset_sizes.values())
    axes[0, 0].set_title('Records per Dataset')
    axes[0, 0].set_xlabel('Dataset')
    axes[0, 0].set_ylabel('Number of Records')
    axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Identifier availability
if identifier_types:
    id_counts = {k: len(v) for k, v in identifier_types.items() if v}
    axes[0, 1].barh(list(id_counts.keys()), list(id_counts.values()))
    axes[0, 1].set_title('Identifier Availability Across Datasets')
    axes[0, 1].set_xlabel('Number of Datasets')

# 3. Missing data percentage
if 'profiles' in data:
    df = data['profiles']
    missing_pct = (df.isna().sum() / len(df)) * 100
    missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=False)
    if len(missing_pct) > 0:
        axes[1, 0].barh(missing_pct.index, missing_pct.values, color='coral')
        axes[1, 0].set_title('Missing Data in Profiles (%)')
        axes[1, 0].set_xlabel('Percentage Missing')

# 4. Temporal distribution (if swipes data available)
if 'swipes' in data:
    df = data['swipes'].copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    daily_counts = df.groupby('date').size()
    axes[1, 1].plot(daily_counts.index, daily_counts.values)
    axes[1, 1].set_title('Daily Swipe Activity')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].set_ylabel('Number of Swipes')
    axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../../data/analysis/dataset_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization saved to: data/analysis/dataset_overview.png")